In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import gdown
import os

# Data Acquisition

In [2]:
# List of all Aircraft Manufacturers and their ICAO codes
URL_MFG_LIST = "https://cfapps.icao.int/doc8643/MnfctrerList.cfm"

mfg_df = pd.read_html(URL_MFG_LIST)[0] \
	.dropna()[1:] \
	.reset_index() \
	.drop(['index'], axis=1)

print(mfg_df.head)

<bound method NDFrame.head of          Manufacturer Code                     Manufacturer Full Name(s)
0     328 SUPPORT SERVICES           328 Support Services GmbH (Germany)
1                   3XTRIM      Zaklady Lotnicze 3Xtrim Sp z oo (Poland)
2                     A-41     Cong Ty Su'A Chu'A May Bay A-41 (Vietnam)
3                      AAC    AAC Amphibiam Airplanes of Canada (Canada)
4                      AAK  Australian Aircraft Kits Pty Ltd (Australia)
...                    ...                                           ...
1521                ZENAIR                           Zenair Ltd (Canada)
1522                ZENITH      ZÃ©nith Aircraft Company (United States)
1523                 ZIVKO         Zivko Aeronautics Inc (United States)
1524                  ZLIN       Moravan Aeroplanes Inc (Czech Republic)
1525         ZLIN AVIATION            Zlin Aviation sro (Czech Republic)

[1526 rows x 2 columns]>


In [3]:
if not os.path.exists("./data"):
	os.mkdir("./data")

mfg_df.to_parquet("./data/mfg.parquet")

In [4]:
# FAA Registered Aircraft Data: Master List
URL_FAA_MASTER = {
	2018 : "1_7-NAfFOoP-etAEAFGNcLvqD5XLutJ5a",
	2019 : "12XOYYGFdgK_FLnWEq0FdVSYfr5PyKANL",
	2020 : "1nT4dQOJIy8MubJz2IH2RsPTVATY4G_0r",
	2021 : "1SbT5XVLPcGzPbskBMtuAN0_OZTHz1Pg5",
	2022 : "1sJ5XZ_cmJQmC7eXnC06Ugh34rb-AF5X8",
}

# FAA Registered Aircraft Data: Aircraft Reference
URL_FAA_ACFREF = {
	2018 : "12VjXZAYC0Qf2TouC5c5o_Tc699_cEVlc",
	2019 : "1QAnf3feFRdo81bc53Ay82YydN-aqcKBT",
	2020 : "1tvR_-6iyjpcgaW58adwJ0t_-2MmoE4ZN",
	2021 : "1N7vzpBUICcxqx7mEKzpU9cawafE82KYR",
	2022 : "1i5U0TK8_otiuEUyHsCmoau9McfL1wsIG"
}

faa_dfs = {}

if not os.path.isdir("./data"):
	os.mkdir("./data")

for year in list(URL_FAA_MASTER.keys()):
	_file_path = f"./data/faa_{year}.parquet"
	_file = Path(_file_path)

	if _file.exists():
	   os.remove(_file_path) # <-- comment out!

	if not _file.exists():
		gdown.download(id=URL_FAA_MASTER[year], output=f"./data/{year}temp.csv")
		gdown.download(id=URL_FAA_ACFREF[year], output=f"./data/{year}temp2.csv")

		_df_master = pd.read_csv(f"./data/{year}temp.csv", dtype="str")
		_df_acfref = pd.read_csv(f"./data/{year}temp2.csv", dtype="str")
		
		_df = _df_master.merge(
			_df_acfref, 
			left_on='MFR MDL CODE', 
			right_on='CODE',
			how='inner',
			suffixes=('__MASTER__', '__ACFREF__')
		)
		
		#for c in _df.columns:
		#	print(c, _df[c].unique())

		_df = _df[['N-NUMBER', 'MFR', 'YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS', 'NO-ENG']]
		for field in ['YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS', 'NO-ENG']:
			_df[field] = _df[field].apply(lambda x: None if str(x).strip() in ['', 'O', 'H'] else int(x))

		_df = _df.dropna()
		
		_df['AGE'] = int(year) - _df['YEAR MFR']
		
		print(_df.columns)

		_df.to_parquet(_file_path, compression="gzip", )

		print("original:", len(_df_master), "| joined: ", len(_df))

		os.remove(f"./data/{year}temp.csv")
		os.remove(f"./data/{year}temp2.csv")


	#faa_dfs[year] = pd.read_parquet(_file_path)
	_df = pd.read_parquet(_file_path)
	print(_df.head())

Downloading...
From (original): https://drive.google.com/uc?id=1_7-NAfFOoP-etAEAFGNcLvqD5XLutJ5a
From (redirected): https://drive.google.com/uc?id=1_7-NAfFOoP-etAEAFGNcLvqD5XLutJ5a&confirm=t&uuid=c3a8762c-7faa-4728-9ec2-7e86dbaa504d
To: /Users/m/Documents/GitHub/CS4403Project/data/2018temp.csv
100%|██████████| 181M/181M [00:07<00:00, 23.2MB/s] 
Downloading...
From: https://drive.google.com/uc?id=12VjXZAYC0Qf2TouC5c5o_Tc699_cEVlc
To: /Users/m/Documents/GitHub/CS4403Project/data/2018temp2.csv
100%|██████████| 7.77M/7.77M [00:00<00:00, 15.3MB/s]


Index(['N-NUMBER', 'MFR', 'YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS',
       'NO-ENG', 'AGE'],
      dtype='object')
original: 294221 | joined:  253798
  N-NUMBER                             MFR  YEAR MFR  SPEED  TYPE-ACFT  \
0    1      GULFSTREAM AEROSPACE              1988.0      0        5.0   
1    100    PIPER                             1940.0     67        4.0   
2    10001  WACO                              1928.0     79        4.0   
3    10002  TEXAS HELICOPTER CORP             1979.0      0        6.0   
7    10006  BEECH                             1955.0      0        4.0   

   NO-SEATS  NO-ENG   AGE  
0        22       2  30.0  
1         2       1  78.0  
2         3       1  90.0  
3         1       1  39.0  
7         4       1  63.0  


Downloading...
From (original): https://drive.google.com/uc?id=12XOYYGFdgK_FLnWEq0FdVSYfr5PyKANL
From (redirected): https://drive.google.com/uc?id=12XOYYGFdgK_FLnWEq0FdVSYfr5PyKANL&confirm=t&uuid=feefb991-f292-406a-b443-f86848a617a6
To: /Users/m/Documents/GitHub/CS4403Project/data/2019temp.csv
100%|██████████| 178M/178M [00:10<00:00, 17.5MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1QAnf3feFRdo81bc53Ay82YydN-aqcKBT
To: /Users/m/Documents/GitHub/CS4403Project/data/2019temp2.csv
100%|██████████| 7.87M/7.87M [00:00<00:00, 14.8MB/s]


Index(['N-NUMBER', 'MFR', 'YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS',
       'NO-ENG', 'AGE'],
      dtype='object')
original: 290100 | joined:  249751
  N-NUMBER                             MFR  YEAR MFR  SPEED  TYPE-ACFT  \
0    1      GULFSTREAM AEROSPACE              1988.0      0        5.0   
1    100    PIPER                             1940.0     67        4.0   
2    10001  WACO                              1928.0     79        4.0   
3    10002  TEXAS HELICOPTER CORP             1979.0      0        6.0   
6    10006  BEECH                             1955.0      0        4.0   

   NO-SEATS  NO-ENG   AGE  
0        22       2  31.0  
1         2       1  79.0  
2         3       1  91.0  
3         1       1  40.0  
6         4       1  64.0  


Downloading...
From (original): https://drive.google.com/uc?id=1nT4dQOJIy8MubJz2IH2RsPTVATY4G_0r
From (redirected): https://drive.google.com/uc?id=1nT4dQOJIy8MubJz2IH2RsPTVATY4G_0r&confirm=t&uuid=94df4ddb-6c51-40bf-811e-f693f5371d78
To: /Users/m/Documents/GitHub/CS4403Project/data/2020temp.csv
100%|██████████| 177M/177M [00:08<00:00, 20.6MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1tvR_-6iyjpcgaW58adwJ0t_-2MmoE4ZN
To: /Users/m/Documents/GitHub/CS4403Project/data/2020temp2.csv
100%|██████████| 13.9M/13.9M [00:00<00:00, 15.9MB/s]


Index(['N-NUMBER', 'MFR', 'YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS',
       'NO-ENG', 'AGE'],
      dtype='object')
original: 287984 | joined:  245127
  N-NUMBER                             MFR  YEAR MFR  SPEED  TYPE-ACFT  \
0    100    PIPER                             1940.0     67        4.0   
1    10001  WACO                              1928.0     79        4.0   
2    10002  TEXAS HELICOPTER CORP             1979.0      0        6.0   
5    10006  BEECH                             1955.0      0        4.0   
6    10007  CESSNA                            1966.0    141        4.0   

   NO-SEATS  NO-ENG   AGE  
0         2       1  80.0  
1         3       1  92.0  
2         1       1  41.0  
5         4       1  65.0  
6         4       1  54.0  


Downloading...
From (original): https://drive.google.com/uc?id=1SbT5XVLPcGzPbskBMtuAN0_OZTHz1Pg5
From (redirected): https://drive.google.com/uc?id=1SbT5XVLPcGzPbskBMtuAN0_OZTHz1Pg5&confirm=t&uuid=aba1abd9-6d8d-4197-932a-4fd7e2fe1e12
To: /Users/m/Documents/GitHub/CS4403Project/data/2021temp.csv
100%|██████████| 177M/177M [00:09<00:00, 18.4MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1N7vzpBUICcxqx7mEKzpU9cawafE82KYR
To: /Users/m/Documents/GitHub/CS4403Project/data/2021temp2.csv
100%|██████████| 14.1M/14.1M [00:00<00:00, 15.8MB/s]


Index(['N-NUMBER', 'MFR', 'YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS',
       'NO-ENG', 'AGE'],
      dtype='object')
original: 287769 | joined:  241916
  N-NUMBER                             MFR  YEAR MFR  SPEED  TYPE-ACFT  \
0    1      CESSNA                            2014.0      0        5.0   
1    100    PIPER                             1940.0     67        4.0   
2    10001  WACO                              1928.0     79        4.0   
4    10006  BEECH                             1955.0      0        4.0   
5    10007  CESSNA                            1966.0    141        4.0   

   NO-SEATS  NO-ENG   AGE  
0         9       2   7.0  
1         2       1  81.0  
2         3       1  93.0  
4         4       1  66.0  
5         4       1  55.0  


Downloading...
From (original): https://drive.google.com/uc?id=1sJ5XZ_cmJQmC7eXnC06Ugh34rb-AF5X8
From (redirected): https://drive.google.com/uc?id=1sJ5XZ_cmJQmC7eXnC06Ugh34rb-AF5X8&confirm=t&uuid=32cb068b-8118-4b66-b5ba-a3613ca6a380
To: /Users/m/Documents/GitHub/CS4403Project/data/2022temp.csv
100%|██████████| 177M/177M [00:09<00:00, 18.2MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1i5U0TK8_otiuEUyHsCmoau9McfL1wsIG
To: /Users/m/Documents/GitHub/CS4403Project/data/2022temp2.csv
100%|██████████| 14.3M/14.3M [00:00<00:00, 18.2MB/s]


Index(['N-NUMBER', 'MFR', 'YEAR MFR', 'SPEED', 'TYPE-ACFT', 'NO-SEATS',
       'NO-ENG', 'AGE'],
      dtype='object')
original: 288389 | joined:  239925
  N-NUMBER                             MFR  YEAR MFR  SPEED  TYPE-ACFT  \
0    1      CESSNA                            2014.0      0        5.0   
1    100    PIPER                             1940.0     67        4.0   
2    10001  WACO                              1928.0     79        4.0   
4    10006  BEECH                             1955.0      0        4.0   
5    10007  CESSNA                            1966.0    141        4.0   

   NO-SEATS  NO-ENG   AGE  
0         9       2   8.0  
1         2       1  82.0  
2         3       1  94.0  
4         4       1  67.0  
5         4       1  56.0  


In [5]:
# Flight Delay Data
URL_DELAY = {
	2018 : "1XWnRQYvx8W2TPgM9ue_exLo2st8YWVHj",
	2019 : "1YJgaB_sONi6nWCBReyEqdOlbq_ojyjWc",
	2020 : "1TCL4hetIDjCjQxqvoEOvGjg7j4D63dtb",
	2021 : "1etYb4mCyM_D-6ITemSaRuz--GDOgtQJf",
	2022 : "1nL4-hrcRLm8mEACuyX1Q8_0H3N04E9Kh",
}


delay_dfs = {}

for year in list(URL_DELAY.keys()):
	_file_path = f"./data/delay_{year}.parquet"
	_file = Path(_file_path)

	if not _file.exists():
		gdown.download(
			id=URL_DELAY[year],
			output=_file_path
		)

	#_df = pd.read_parquet(_file_path)
	#print(_df.head())

In [6]:
delay_2018_df = pd.read_parquet("./data/delay_2018.parquet").dropna()
delay_2018_df[['Cancelled', 'Diverted']].describe()

,Cancelled,Diverted
count,5578618,5578618
unique,1,1
top,False,False
freq,5578618,5578618


In [7]:
delay_2018_df['Diverted'].unique()

array([False])

In [8]:
delay_2018_df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [9]:
print("Dataframe size (MB):", end=" ")
print(delay_2018_df.memory_usage(index=1).sum() // 1000000)

Dataframe size (MB): 2688


In [10]:
delay_2018_df = delay_2018_df[['Cancelled', 'Diverted', 'DepDelayMinutes', 'ArrDelayMinutes',  'Tail_Number', 'OriginAirportID', 'DestAirportID']]
delay_2018_df.columns

Index(['Cancelled', 'Diverted', 'DepDelayMinutes', 'ArrDelayMinutes',
       'Tail_Number', 'OriginAirportID', 'DestAirportID'],
      dtype='object')

In [11]:
print("Dataframe size (MB):", end=" ")
print(delay_2018_df.memory_usage(index=1).sum() // 1000000)

Dataframe size (MB): 278


In [12]:
faa_2018_df = pd.read_parquet('./data/faa_2018.parquet')

print("Dataframe size (MB): ", faa_2018_df.memory_usage(index=1).sum() // 1000000)

pd.options.display.max_columns = 1000
faa_2018_df.head()

Dataframe size (MB):  18


,N-NUMBER,MFR,YEAR MFR,SPEED,TYPE-ACFT,NO-SEATS,NO-ENG,AGE
0,1,GULFSTREAM AEROSPACE,1988.0,0,5.0,22,2,30.0
1,100,PIPER,1940.0,67,4.0,2,1,78.0
2,10001,WACO,1928.0,79,4.0,3,1,90.0
3,10002,TEXAS HELICOPTER CORP,1979.0,0,6.0,1,1,39.0
7,10006,BEECH,1955.0,0,4.0,4,1,63.0


In [14]:
import airportsdata

delay_2018_df.head()


,Cancelled,Diverted,DepDelayMinutes,ArrDelayMinutes,Tail_Number,OriginAirportID,DestAirportID
0,False,False,0.0,0.0,N8928A,10146,10397
1,False,False,0.0,0.0,N800AY,10146,10397
2,False,False,0.0,0.0,N8836A,10146,10397
3,False,False,0.0,0.0,N800AY,10146,10397
4,False,False,0.0,0.0,N8839E,10146,10397
